In [10]:
import pandas as pd
df = pd.read_csv("/Users/iyeeun/Documents/coreview_3/data/df_sumWordSentence.csv")
df

,이름,작성일자,평점,리뷰 내용,맛 만족도,preprocessed_review,krwordrank
0,박*혜,2025.01.14,5,"짜파게티 마라, 어떤 제품일까?먼저 짜파게티 마라의 기본 정보를 알아보았어요. 이 ...",아주 넉넉해요,짜파게티 마라 어떤 제품 일까먼저 짜파게티 마라 기본 정보를 알아보았어요 제품 유명...,['마라']
1,dimss,2025.01.13,4,안녕하세요!! 리뷰하는 dimss입니다^^오늘도 솔직 사용후기 들고 왔어요❤️⬇️⬇...,적당히 남았어요,안녕하세요 리뷰 하는 입니다 오늘도 솔직 사용후기 들고 왔어요짜파게티 마라 솔직 후...,"['마라', '분들', '짜파']"
2,이*람,2025.01.12,5,### 구매 계기매운 음식을 좋아하는 저는 최근 마라탕이나 마라샹궈와 같은 마라 요...,적당히 남았어요,구매 계기매운 음식 좋아 하는 저는 최근 마라 탕이나 마라 샹궈와 같은 마라 요리 ...,"['마라', '매운', '맛이', '정말', '국물']"
3,시마에나가,2025.01.11,4,"⏹️ 요약 ⏹️✅ 구성 - 면, 후레이크, 짜장스프, 마라조미유✅ 맛 - 맵찔이 먹...",적당히 남았어요,요약 구성 후레이크 짜장스프 마라 조미유 맵찔이 먹으면 안됨 그냥 매운 마라 구매 ...,"['마라', '넣고', '짜파', '먹어']"
4,김*수,2025.01.11,5,짜파게티 마라를 구매한 후기를 이모티콘을 활용하여 작성해 보겠습니다. 아래와 같은 ...,아주 넉넉해요,짜파게티 마라 구매 후기 이모티콘을 활용 하여 작성해 보겠습니다 아래와 같은 방식 ...,"['마라', '매운', '맛이']"
...,...,...,...,...,...,...,...
259,잔잔다에,2025.01.01,1,처음 입에 넣을 때 거부감 드는 향과 뒤에 매운 맛만 느껴짐참고서라도 먹고싶지 않은 맛,맛 평가 없음,NaN,NaN
260,코만도x,2025.01.10,1,맛과 어울리지도 않게 맵기만함고급스러운 매운맛이 아니고 싼매운맛. 유통기한 끝나가는...,적당히 남았어요,NaN,NaN
261,sssgg,2025.01.04,1,마라향이 뭔가 조화롭지않아서 좀 인위적인?느낌의 향만나고우선 진짜 너무 맵기만해요,적당히 남았어요,NaN,NaN
262,쿠리사,2025.01.13,1,짜파게티도 맛있고 블랙도 맛있는데이건 영.....마라향이 확 나는게 아니라서 이상함...,아주 넉넉해요,NaN,NaN


## 평점 평균 계산

In [12]:
import pandas as pd

def calculate_keyword_average_ratings(dataframe, keywords, review_column, rating_column):
    keyword_scores = {}
    for keyword in keywords:
        # Filter reviews that contain the keyword
        keyword_reviews = dataframe[dataframe[review_column].str.contains(keyword, na=False)]
        # Calculate the average rating
        avg_score = keyword_reviews[rating_column].mean()
        keyword_scores[keyword] = avg_score

    # Convert the results to a DataFrame
    result_df = pd.DataFrame(list(keyword_scores.items()), columns=['Keyword', 'Average Rating'])
    return result_df


keywords = ['마라', '짜파게티', '매운']  # keywordrank로 뽑아낸 k개 단어들의 리스트
result_df = calculate_keyword_average_ratings(df, keywords, '리뷰 내용', '평점')
result_df

,Keyword,Average Rating
0,마라,4.371560
1,짜파게티,4.506579
2,매운,4.579832


## 1. extract sentence

In [15]:
import pandas as pd
import re
import random

# 마침표가 없어서 문장split이 안되길래 문장 별로 끊을 수 있도록 함수 추가함
def split_sentences(text):
    return re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', text)

def get_sentences_with_keyword(review, keywords):
    sentences = split_sentences(review)
    return [sentence for sentence in sentences if any(keyword in sentence for keyword in keywords)]

def contains_numbers(text):
    return bool(re.search(r'\d+', text))

def filter_sentences(df, keywords, min_length=30, max_length=60, num_samples=3):
    df["filtered_sentences"] = df["리뷰 내용"].apply(lambda x: get_sentences_with_keyword(str(x), keywords))

    all_sentences = [
        sentence
        for sentences in df["filtered_sentences"].dropna()
        for sentence in sentences
        if not contains_numbers(sentence) 
    ]

    # 문장 길이가 30자 이상 50자 이하인 문장만 필터링함
    filtered_sentences = [
        sentence for sentence in all_sentences if min_length <= len(sentence) <= max_length
    ]

    if len(filtered_sentences) >= num_samples:
        return random.sample(filtered_sentences, num_samples)  # 그냥 랜덤으로 문장 선택
    else:
        return filtered_sentences 


keywords = result_df['Keyword'][0]
result = filter_sentences(df, keywords, min_length=30, max_length=60, num_samples=3)

result

['다만 저한테는 마라 맛이 조금 더 진하면좋겠다~~ 정도인데요.',
 '이 제품은 유명한 한국의 라면 브랜드가 만들어낸 것으로, 매콤한 맛이 특징이에요.',
 '짜파게티는 본래의 짜장면 맛이 좋아서 자주 먹곤 했는데, 마라맛까지 추가되니 정말 궁금해졌습니다.']

## 2. summary review w/ kobart

#### ngram

In [6]:
import pandas as pd
from itertools import islice

def generate_ngrams(text, n=3):
    """
    Generate n-grams from the given text.

    Parameters:
        text (str): The input text to process.
        n (int): The size of the n-grams to generate.

    Returns:
        list: A list of n-grams.
    """
    if pd.isna(text):
        return []
    words = text.split()
    ngrams = [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]
    return ngrams

def extract_ngrams_with_keyword(data, keyword, review_column='preprocessed_review', window=2):
    """
    Extract n-grams containing a specific keyword, including words before and after the keyword.

    Parameters:
        data (pd.DataFrame): The input DataFrame.
        keyword (str): The keyword to search for in the n-grams.
        review_column (str): The column name containing the reviews.
        window (int): The number of words to include before and after the keyword.

    Returns:
        pd.DataFrame: A DataFrame with the original review and the extracted keyword-specific n-grams.
    """
    def extract_context_ngrams(text):
        if pd.isna(text):
            return []
        words = text.split()
        result = []
        for i, word in enumerate(words):
            if keyword in word:
                # Extract words within the window before and after the keyword
                start = max(i - window, 0)
                end = min(i + window + 1, len(words))
                result.append(' '.join(words[start:end]))
        return result

    data['filtered_ngrams'] = data[review_column].apply(extract_context_ngrams)
    result = data[[review_column, 'filtered_ngrams']]
    return result

# Example usage (assuming df is the DataFrame and contains 'preprocessed_review')
result = extract_ngrams_with_keyword(df, keyword='마라', review_column='리뷰 내용', window=2)
# result = extract_ngrams_with_keyword(df, keyword='마라', review_column='preprocessed_review', window=2)
result

,리뷰 내용,filtered_ngrams
0,"짜파게티 마라, 어떤 제품일까?먼저 짜파게티 마라의 기본 정보를 알아보았어요. 이 ...","[짜파게티 마라, 어떤 제품일까?먼저, 제품일까?먼저 짜파게티 마라의 기본 정보를,..."
1,안녕하세요!! 리뷰하는 dimss입니다^^오늘도 솔직 사용후기 들고 왔어요❤️⬇️⬇...,"[들고 왔어요❤️⬇️⬇️⬇️✨짜파게티 마라 140g 4개,, 19일까지✅구매 이유-..."
2,### 구매 계기매운 음식을 좋아하는 저는 최근 마라탕이나 마라샹궈와 같은 마라 요...,"[저는 최근 마라탕이나 마라샹궈와 같은, 최근 마라탕이나 마라샹궈와 같은 마라, 마..."
3,"⏹️ 요약 ⏹️✅ 구성 - 면, 후레이크, 짜장스프, 마라조미유✅ 맛 - 맵찔이 먹...","[후레이크, 짜장스프, 마라조미유✅ 맛 -, 그냥 매운 마라맛⏹️ 구매 동기, 불닭..."
4,짜파게티 마라를 구매한 후기를 이모티콘을 활용하여 작성해 보겠습니다. 아래와 같은 ...,"[짜파게티 마라를 구매한 후기를, 있습니다: 짜파게티 마라를 드디어 구매해봤어요!,..."
...,...,...
259,처음 입에 넣을 때 거부감 드는 향과 뒤에 매운 맛만 느껴짐참고서라도 먹고싶지 않은 맛,[]
260,맛과 어울리지도 않게 맵기만함고급스러운 매운맛이 아니고 싼매운맛. 유통기한 끝나가는...,[]
261,마라향이 뭔가 조화롭지않아서 좀 인위적인?느낌의 향만나고우선 진짜 너무 맵기만해요,[마라향이 뭔가 조화롭지않아서]
262,짜파게티도 맛있고 블랙도 맛있는데이건 영.....마라향이 확 나는게 아니라서 이상함...,"[블랙도 맛있는데이건 영.....마라향이 확 나는게, 너무 없음 ㅠㅠ마라향기름 안넣..."


In [7]:
import pandas as pd
import random
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load tokenizer and model for summarization
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')


# Function to generate n-grams from a text
def generate_ngrams(text, n=3):
    if pd.isna(text):
        return []
    words = text.split()
    ngrams = [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]
    return ngrams

# Function to extract n-grams containing a specific keyword
def extract_ngrams_with_keyword(data, keyword, review_column='preprocessed_review', n=5):
    # Generate n-grams for each review
    data['ngrams'] = data[review_column].apply(lambda x: generate_ngrams(x, n))
    
    # Filter n-grams containing the keyword
    data['filtered_ngrams'] = data['ngrams'].apply(lambda ngrams: [ngram for ngram in ngrams if keyword in ngram])
    return data

# Function to preprocess text before summarization
def preprocess_text(text):
    # Remove duplicate words and limit excessive repetition
    words = text.split()
    deduplicated = []
    for word in words:
        if len(deduplicated) < 2 or deduplicated[-2:] != [word, word]:
            deduplicated.append(word)
    return ' '.join(deduplicated)

# Function to summarize text using KoBART
def summarize_text_kobart(text, max_length=50, min_length=30):
    if not text or len(text.strip()) == 0:
        return None
    # Preprocess text to remove duplicates
    text = preprocess_text(text)
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=1.0,
        num_beams=4,
        repetition_penalty=4.0,  # Higher penalty for repetitive outputs
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main function to process and summarize the dataset
def process_and_summarize(df, keyword, n=5, samples_per_rating=3):
    # Extract n-grams with keyword
    df = extract_ngrams_with_keyword(df, keyword, review_column='preprocessed_review', n=n)

    # Create a new DataFrame to store results
    intermediate_results = []

    # Process each rating
    for rating in sorted(df['평점'].unique(), reverse=True):
        rating_df = df[df['평점'] == rating]
        all_ngrams = [ngram for ngrams in rating_df['filtered_ngrams'] for ngram in ngrams if ngram.strip() != ""]

        if not all_ngrams:  # Skip if no valid n-grams are found
            continue

        # Select the longest n-grams
        longest_ngrams = sorted(all_ngrams, key=len, reverse=True)[:samples_per_rating]

        # Summarize each longest n-gram
        summarized_ngrams = [summarize_text_kobart(ngram) for ngram in longest_ngrams]

        # Combine the summarized n-grams into one text
        combined_text = ' '.join(summarized_ngrams)

        # Summarize the combined text
        summary = summarize_text_kobart(combined_text, max_length=50, min_length=10)

        # Store intermediate results
        intermediate_results.append({
            '평점': rating,
            'summary': summary
        })

    # Create a final summary combining all summaries
    combined_summaries = ' '.join([result['summary'] for result in intermediate_results if result['summary']])
    final_summary = summarize_text_kobart(combined_summaries, max_length=50, min_length=10)

    # Append the final summary to the results
    intermediate_results.append({
        '평점': 'Final',
        'summary': final_summary
    })

    return pd.DataFrame(intermediate_results)

# Example usage
keyword = '짜파'
result_df = process_and_summarize(df, keyword)
result_df


/opt/homebrew/Caskroom/miniconda/base/envs/coreview/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


,평점,summary
0,5,짜파게티를 먹을 친정아빠가 항상 고춧가루를뿌려드시던 고춧가루를 뿌려드는데 최애라면인...
1,4,기본짜파게티의 고소고소요없고 마유좋아하시는 분들은 환장하실맛입니다
2,3,사전예약 주문이라서 일주일이상걸렸네요
3,2,사천짜파게티 흔적은 마라볶음면에 짜파게티를 맵습니다 마라볶음면에 짜파게티가 보다 보...
4,Final,짜파게티를 먹을 친정아빠가 항상 고춧가루를뿌려드시던 고춧가루를 뿌려드는데 기본짜파게...
